# __Dense matrix tests__

## Outline
- __Dependencies__
- __Dynamic ansatz__
    - Noiseless
    - With noise using Qiskit AER
- __Static ansatz__
    - Noiseless
    - With noise using Qiskit AER

In [6]:
import numpy as np
from device_var_lse_solver import DeviceVarLSESolver
from device import DeviceType
from conditioned_matrix import conditionedMatrix
from non_dynamical_ansatz import fixed_layered_ansatz
from tqdm import tqdm
from device import Device

qubits = 4
b = np.ones(2**qubits)/np.sqrt(2**qubits)
depth=10

assert qubits % 2 == 0 # Needed for Poisson matrix

In [7]:
# As defined by [10]
def TRC_ADA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += d*z
    return trc

def TRC_ASA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += z*depth
    return trc

## __Dynamic ansatz__

In [ ]:
# For poisson matrix
N_runs = 10
N_steps = 1500
redo_calc = 5

cs = [1.04e+00, 1.49e+00, 1.58e+02]
cs = [10, 50, 100]

local_noiseless_dynamic_trc = np.zeros((redo_calc, len(cs)))
global_noiseless_dynamic_trc = np.zeros((redo_calc, len(cs)))
local_noise_dynamic_trc = np.zeros((redo_calc, len(cs)))
global_noise_dynamic_trc = np.zeros((redo_calc, len(cs)))

local_noiseless_static_trc = np.zeros((redo_calc, len(cs)))
global_noiseless_static_trc = np.zeros((redo_calc, len(cs)))
local_noise_static_trc = np.zeros((redo_calc, len(cs)))
global_noise_static_trc = np.zeros((redo_calc, len(cs)))

local_noise_dynamic_error = np.zeros((redo_calc, len(cs)))
local_noiseless_dynamic_error = np.zeros((redo_calc, len(cs)))
local_noise_static_error = np.zeros((redo_calc, len(cs)))
local_noiseless_static_error = np.zeros((redo_calc, len(cs)))

global_noise_dynamic_error = np.zeros((redo_calc, len(cs)))
global_noiseless_dynamic_error = np.zeros((redo_calc, len(cs)))
global_noise_static_error = np.zeros((redo_calc, len(cs)))
global_noiseless_static_error = np.zeros((redo_calc, len(cs)))

local_noise_dynamic_solution = np.zeros((redo_calc, len(cs), 2**qubits))
local_noiseless_dynamic_solution = np.zeros((redo_calc, len(cs), 2**qubits))
local_noise_static_solution = np.zeros((redo_calc, len(cs), 2**qubits))
local_noiseless_static_solution = np.zeros((redo_calc, len(cs), 2**qubits))

global_noise_dynamic_solution = np.zeros((redo_calc, len(cs), 2**qubits))
global_noiseless_dynamic_solution = np.zeros((redo_calc, len(cs), 2**qubits))
global_noise_static_solution = np.zeros((redo_calc, len(cs), 2**qubits))
global_noiseless_static_solution = np.zeros((redo_calc, len(cs), 2**qubits))

correct_solutions = np.zeros((redo_calc, len(cs), 2**qubits))


device = Device(DeviceType.QISKIT_AER, qubits=qubits)

param_shape = (qubits + depth*(qubits+qubits -2),)

pbar = tqdm(range(redo_calc))
for i in pbar:
        for j, c in enumerate(cs):
                a = conditionedMatrix(2**qubits, c)
                classical_solution = np.linalg.solve(a, b)
                normalized_classical_solution = np.square(classical_solution / np.linalg.norm(classical_solution))
                normalized_classical_solution /= np.linalg.norm(normalized_classical_solution)

                correct_solutions[i,j] = np.abs(normalized_classical_solution)

                ###################################
                #####    DYNAMIC NOISELESS    #####
                ###################################
                lse_local = DeviceVarLSESolver(a,
                        b, 
                        method='direct',
                        local=True, 
                        lr=0.1, 
                        steps=N_steps,
                        threshold=0.001, 
                        epochs=N_runs,
                        silent=True)
                
                lse_global = DeviceVarLSESolver(a,
                        b, 
                        method='direct',
                        local=False, 
                        lr=0.1, 
                        steps=N_steps,
                        threshold=0.001, 
                        epochs=N_runs,
                        silent=True)
        
                pbar.set_description(f"Local Dynamic Noiseless ({1+j*8}/{len(cs)*8})")
                solution_local, param_local, it_count_local = lse_local.solve()
                pbar.set_description(f"Global Dynamic Noiseless ({2+j*8}/{len(cs)*8})")
                solution_global, param_global, it_count_global = lse_global.solve()
                solution_local /= np.linalg.norm(solution_local)
                solution_global /= np.linalg.norm(solution_global)

                local_noiseless_dynamic_solution[i, j] = solution_local
                global_noiseless_dynamic_solution[i, j] = solution_global

                local_noiseless_dynamic_trc[i, j] = TRC_ADA(it_count_local)
                global_noiseless_dynamic_trc[i, j] = TRC_ADA(it_count_global)

                local_noiseless_dynamic_error[i, j] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
                global_noiseless_dynamic_error[i, j] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

                ###################################
                #####    DYNAMIC NOISEFULL    #####
                ###################################
                lse_local = DeviceVarLSESolver(a,
                        b, 
                        method='direct',
                        local=True, 
                        lr=0.1, 
                        steps=N_steps,
                        threshold=0.001, 
                        epochs=N_runs,
                        device=device,
                        silent=True)
                
                lse_global = DeviceVarLSESolver(a,
                        b, 
                        method='direct',
                        local=False, 
                        lr=0.1, 
                        steps=N_steps,
                        threshold=0.001, 
                        epochs=N_runs,
                        device=device,
                        silent=True)
        
                pbar.set_description(f"Local Dynamic Noise ({3+j*8}/{len(cs)*8})")
                solution_local, param_local, it_count_local = lse_local.solve()
                pbar.set_description(f"Global Dynamic Noise ({4+j*8}/{len(cs)*8})")
                solution_global, param_global, it_count_global = lse_global.solve()
                solution_local /= np.linalg.norm(solution_local)
                solution_global /= np.linalg.norm(solution_global)

                local_noise_dynamic_solution[i, j] = solution_local
                global_noise_dynamic_solution[i, j] = solution_global

                local_noise_dynamic_trc[i, j] = TRC_ADA(it_count_local)
                global_noise_dynamic_trc[i, j] = TRC_ADA(it_count_global)

                local_noise_dynamic_error[i, j] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
                global_noise_dynamic_error[i, j] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

                ###################################
                #####    STATIC  NOISELESS    #####
                ###################################
                lse_local = DeviceVarLSESolver(a,
                        b, 
                        method='direct',
                        ansatz=fixed_layered_ansatz,
                        weights=param_shape,
                        local=True, 
                        lr=0.1, 
                        steps=N_steps,
                        threshold=0.001, 
                        epochs=N_runs,
                        silent=True)
                
                lse_global = DeviceVarLSESolver(a,
                        b, 
                        method='direct',
                        ansatz=fixed_layered_ansatz,
                        weights=param_shape,
                        local=False, 
                        lr=0.1, 
                        steps=N_steps,
                        threshold=0.001, 
                        epochs=N_runs,
                        silent=True)
        
                pbar.set_description(f"Local Static Noiseless ({5+j*8}/{len(cs)*8})")
                solution_local, param_local, it_count_local = lse_local.solve()
                pbar.set_description(f"Global Static Noiseless ({6+j*8}/{len(cs)*8})")
                solution_global, param_global, it_count_global = lse_global.solve()
                solution_local /= np.linalg.norm(solution_local)
                solution_global /= np.linalg.norm(solution_global)

                local_noiseless_static_solution[i, j] = solution_local
                global_noiseless_static_solution[i, j] = solution_global

                local_noiseless_static_trc[i, j] = TRC_ASA(it_count_local)
                global_noiseless_static_trc[i, j] = TRC_ASA(it_count_global)

                local_noiseless_static_error[i, j] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
                global_noiseless_static_error[i, j] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

                ###################################
                #####    STATIC  NOISEFULL    #####
                ###################################
                lse_local = DeviceVarLSESolver(a,
                        b, 
                        method='direct',
                        local=True, 
                        ansatz=fixed_layered_ansatz,
                        weights=param_shape,
                        lr=0.1, 
                        steps=N_steps,
                        threshold=0.001, 
                        epochs=N_runs,
                        device=device,
                        silent=True)
                
                lse_global = DeviceVarLSESolver(a,
                        b, 
                        method='direct',
                        local=False, 
                        ansatz=fixed_layered_ansatz,
                        weights=param_shape,
                        lr=0.1, 
                        steps=N_steps,
                        threshold=0.001, 
                        epochs=N_runs,
                        device=device,
                        silent=True)
        
                pbar.set_description(f"Local Static Noise ({7+j*8}/{len(cs)*8})")
                solution_local, param_local, it_count_local = lse_local.solve()
                pbar.set_description(f"Global Static Noise ({8+j*8}/{len(cs)*8})")
                solution_global, param_global, it_count_global = lse_global.solve()
                solution_local /= np.linalg.norm(solution_local)
                solution_global /= np.linalg.norm(solution_global)

                local_noise_static_solution[i, j] = solution_local
                global_noise_static_solution[i, j] = solution_global

                local_noise_static_trc[i, j] = TRC_ASA(it_count_local)
                global_noise_static_trc[i, j] = TRC_ASA(it_count_global)

                local_noise_static_error[i, j] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))
                global_noise_static_error[i, j] = np.linalg.norm(solution_global - np.abs(normalized_classical_solution))

  0%|          | 0/5 [00:00<?, ?it/s]

Local Dynamic Noiseless (1/24):   0%|          | 0/5 [00:12<?, ?it/s]


KeyboardInterrupt: 

## __Save vals__

In [ ]:
## Save the vals
filename = f"data/result_dense_test_3"
saved_dict = {
    # Noise dynamic
    "local_noise_dynamic_trc" : local_noise_dynamic_trc,
    "local_noise_dynamic_error" : local_noise_dynamic_error,
    "local_noise_dynamic_solution" : local_noise_dynamic_solution,
    "global_noise_dynamic_trc" : global_noise_dynamic_trc,
    "global_noise_dynamic_error" : global_noise_dynamic_error,
    "global_noise_dynamic_solution" : global_noise_dynamic_solution,

    # Noiseless dynamic
    "local_noiseless_dynamic_trc" : local_noiseless_dynamic_trc,
    "local_noiseless_dynamic_error" : local_noiseless_dynamic_error,
    "local_noiseless_dynamic_solution" : local_noiseless_dynamic_solution,
    "global_noiseless_dynamic_trc" : global_noiseless_dynamic_trc,
    "global_noiseless_dynamic_error" : global_noiseless_dynamic_error,
    "global_noiseless_dynamic_solution" : global_noiseless_dynamic_solution,

    # Noise static
    "local_noise_static_trc" : local_noise_static_trc,
    "local_noise_static_error" : local_noise_static_error,
    "local_noise_static_solution" : local_noise_static_solution,
    "global_noise_static_trc" : global_noise_static_trc,
    "global_noise_static_error" : global_noise_static_error,
    "global_noise_static_solution" : global_noise_static_solution,

    # Noiseless static
    "local_noiseless_static_trc" : local_noiseless_static_trc,
    "local_noiseless_static_error" : local_noiseless_static_error,
    "local_noiseless_static_solution" : local_noiseless_static_solution,
    "global_noiseless_static_trc" : global_noiseless_static_trc,
    "global_noiseless_static_error" : global_noiseless_static_error,
    "global_noiseless_static_solution" : global_noiseless_static_solution,

    "normalized_classical_solution" : normalized_classical_solution
}
np.savez(filename, **saved_dict)
print(f"Saved {filename}")

Saved data/result_dense_test_3


In [ ]:
data = np.load(f"{filename}.npz")

local_noise_dynamic_error = data["local_noise_dynamic_error"]
print(local_noise_dynamic_error)

[[0.99537388 0.98064547 1.21040903]]
